In [1]:
import json

from pitchfilter.pitchfilter import PitchPostFilter
from tonicidentifier.tonicidentifier import TonicLastNote
from adaptivetuning.tuner.tuner import Tuner
from adaptivetuning.synthesizer.synth_symbtr import synth_symbtr

In [2]:
# loading pitch for adaptive tuning
pitch = json.load(open("sampledata/huseyni--sazsemaisi--aksaksemai----tatyos_efendi/8b8d697b-cad9-446e-ad19-5e85a36aa253.json", 'r'))['pitch']

# Post process the pitch track to get rid of spurious pitch estimations and correct octave errors
flt = PitchPostFilter()
pitch = flt.run(pitch)

In [3]:
# identification of the tonic for the related recording of SymbTr

tnc = TonicLastNote()
tonic, pitch, pitch_chunks, pitch_distribution, stable_pitches = tnc.identify(pitch)

# Adapting the tuning and synthesizing the SymbTr

This part synthesizes the score according to the related audio recording.
Extracted stable pitches are used as reference in synth function.

**It takes a time w.r.t. the length of given SymbTr.**
If you want to obseve the status of the process, give *"verbose"* flag as *true*.

Possible synth types are *'sine'* and *'karplus'*
*'Sine'* option takes less time.

In [4]:
adapt = Tuner()
theoretical_histogram, adapted_histogram = adapt.adapt_score_frequencies(musicxml_path="sampledata/huseyni--sazsemaisi--aksaksemai----tatyos_efendi/huseyni--sazsemaisi--aksaksemai----tatyos_efendi.xml",
                                                                         performed_tonic=tonic['value'],
                                                                         stable_pitches=stable_pitches,
                                                                         type='karplus',
                                                                         verbose=False)